In [6]:
# from openai import OpenAI
# from dotenv import load_dotenv
# load_dotenv()
# client = OpenAI()
#
# response = client.responses.create(
#     model="gpt-5-nano",
#     input="Write a one-sentence bedtime story about a unicorn."
# )
#
# print(response.output_text)

In [5]:
from google import genai
from dotenv import load_dotenv
load_dotenv()
import os

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()
text = "Machine Learning is a field of artificial intelligence that enables computers to learn from data and make predictions"
prompt = f"""Explain the key concepts of the text delimited by triple backticks in simple terms ```{text}```"""

print(prompt, '\n')
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=prompt
)
print(response.text)


Explain the key concepts of the text delimited by triple backticks in simple terms ```Machine Learning is a field of artificial intelligence that enables computers to learn from data and make predictions``` 

Okay, let's break down that sentence:

1.  **Machine Learning:** This is the main topic. It's a method or a way of doing things.
2.  **Artificial Intelligence (AI):** This is the bigger, overall goal – making computers smart, like humans. Machine Learning is a *part* of this bigger goal.
3.  **Computers learn from data:** Instead of being told every single rule, the computer looks at lots of examples and information ("data"). From these examples, it figures out patterns and rules on its own.
4.  **Make predictions:** Once the computer has learned from the data, it can then use that knowledge to guess what might happen next, or identify something it hasn't seen before.

**In simple terms:**

Machine Learning is a way to make computers smart (which is called Artificial Intelligence)

In [8]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents='Translate the following English sentence into French: "The weather is nice today"'
)
print(response.text)

The most common and natural way to say "The weather is nice today" in French is:

**Il fait beau aujourd'hui.**

*   "Il fait beau" is a standard impersonal expression meaning "the weather is nice/beautiful."
*   "aujourd'hui" means "today."


#### Zero Shot Prompting: No Examples

In [6]:
chat = client.chats.create(
    model="gemini-2.5-flash",
    config= genai.types.GenerateContentConfig(
        # system_instruction=mr_chris_persona,
        temperature=0.
    )
)
response = chat.send_message('Translate the following English sentence into French: "The weather is nice today')

print(response.text)

**Il fait beau aujourd'hui.**


#### One Shot Prompting: Single Example in prompt

In [10]:
prompt = """Detect the Language of the following sentences:
text: "A plus tard" -> language: "French"
Now detect:
text: "Gracias"
"""

chat = client.chats.create(
    model="gemini-2.5-flash",
    config= genai.types.GenerateContentConfig(
        temperature=0.
    )
)
response = chat.send_message(prompt)

print(response.text)

language: "Spanish"


#### Few-shot prompting

In [11]:
prompt ="""Determine the sentiment of the following sentences:
text: "I love this product! It works perfectly." -> classification: Positive
text: "The service was terrible. I'm very disappointed." -> Classification: Negative
text: "The food was okay, nothing special." -> Classification: Neutral
Now analyze this sentence:
"The movie was amazing, I enjoyed every moment of it!"
"""

response = chat.send_message(prompt)

print(response.text)

Classification: Positive


#### Multi-step prompting

In [12]:
text = "Marie Curie discovered radium 1898"
prompt = f"""You will be provided with a text delimited by triple backticks.
Step 1: Identify the key entities in the sentence.
Step 2:
Classify each entity as a person, event, or object.
```{text}```
"""
response = chat.send_message(prompt)

print(response.text)

Step 1: Identify the key entities in the sentence.
- Marie Curie
- radium
- 1898

Step 2: Classify each entity as a person, event, or object.
- Marie Curie: Person
- radium: Object
- 1898: Event


#### Chain of thought prompting

In [13]:
prompt = "John has 3 apples. He buys 5 more and gives away 2. How many apples does he have now? Think step by step."

response = chat.send_message(prompt)

print(response.text)

Here's how to solve it step-by-step:

1.  **Start with:** John has 3 apples.
2.  **Buys more:** He buys 5 more, so 3 + 5 = 8 apples.
3.  **Gives away:** He gives away 2, so 8 - 2 = 6 apples.

John has 6 apples now.


#### Self Consistency Prompting

In [14]:
instruction = "Imagine three completely independent experts who reason differently are answering this question. The final answer is obtained by majority vote. The question is: "
question = "A bookstore has 20 books ona shelf. A customer buys 5 books. Then, the store restocks with double the number of books bought. After that, another customer buys 8 books. How many books are left on the shelf?"
prompt = instruction + question

response = chat.send_message(prompt)

print(response.text)

Here's how three independent experts might reason:

**Expert 1: The Sequential Tracker**

"I'll follow the books on the shelf step-by-step, updating the total after each event."

1.  **Start:** The shelf has 20 books.
2.  **First purchase:** A customer buys 5 books.
    *   Current books: 20 - 5 = 15 books.
3.  **Restock:** The store restocks with double the number of books bought (2 * 5 = 10 books).
    *   Current books: 15 + 10 = 25 books.
4.  **Second purchase:** Another customer buys 8 books.
    *   Current books: 25 - 8 = 17 books.

*Expert 1's Answer: 17 books*

---

**Expert 2: The Transaction Analyst**

"I'll view this as a series of transactions affecting the initial quantity, calculating the net effect."

1.  **Initial state:** 20 books.
2.  **Transaction 1 (Purchase):** - 5 books.
3.  **Transaction 2 (Restock):** The number bought was 5, so double that is 10 books. This is an addition: + 10 books.
4.  **Transaction 3 (Purchase):** - 8 books.

Now, let's apply these changes

In [23]:
system_prompt = "Act as an experienced hiring manager. Ask me five interview questions one by one. After I respond, provide constructive feedback on my answer, including strengths and areas for improvement. If my response is incomplete, guide me toward a better answer."
user_prompt = "I'm preparing for a Data Scientist interview. Can you ask me fine five questions and evaluate my responses?"

chat = client.chats.create(
    model="gemini-2.5-flash",
    config= genai.types.GenerateContentConfig(
        system_instruction = system_prompt,
        temperature=0.
    )
)
response = chat.send_message(user_prompt)
print(response.text)
while True:
    print(f"User is Speaking \n")
    user_input = input("\nYou: ")
    response = chat.send_message(user_input)
    print(f"Gemini is Speaking: \n{response.text}")

    break
print(response.text)

Excellent! It's great you're proactively preparing. As an experienced hiring manager, I'm here to help you sharpen your responses.

Let's start with our first question. Take your time, and remember to be concise yet comprehensive.

**Question 1:** "Tell me about yourself and what led you to pursue a career in Data Science."
User is Speaking 

Gemini is Speaking: 
Understood. While "Tell me about yourself" is a very common opening, we can certainly move on.

Let's jump right into a more technical scenario.

**Question 2:** "Describe a challenging data science project you've worked on. What was the problem, what was your approach, what were the key challenges, and what was the outcome?"
User is Speaking 

Gemini is Speaking: 
Okay, I understand you'd like to skip these broader, more experience-based questions.

As a hiring manager, I'd typically use questions like "Tell me about yourself" and "Describe a challenging project" to understand your communication style, your ability to structu

KeyboardInterrupt: Interrupted by user

In [9]:
import json
user_profile = {
"name": "Zara",
"interest": "coding",
"goal": "learn AI",
"tone": "friendly",
"length": "short"
}
prompt = f"""My name is {user_profile['name']}. I love {user_profile['interest']} and want to
{user_profile['goal']}. Write a {user_profile['tone']} and {user_profile['length']} guide on how I can
start."""
print("Generated Prompt: \n", prompt)

Generated Prompt: 
 My name is Zara. I love coding and want to
learn AI. Write a friendly and short guide on how I can
start.


In [10]:
chat = client.chats.create(
    model="gemini-2.5-flash",
    config= genai.types.GenerateContentConfig(
        # system_instruction = prompt,
        temperature=0
    )
)
response = chat.send_message(prompt)

print(response.text)

Hi Zara! That's fantastic! With your coding background, you're already in a great spot to dive into AI. Here's a friendly and short guide to get you started:

1.  **Leverage Your Python Skills:** Python is the absolute go-to language for AI. If you're already comfortable with it, you're ahead of the game! If not, a quick refresher on Python fundamentals (especially data structures and functions) will be super helpful.

2.  **Understand the Basics of Machine Learning (ML):** AI is a broad field, and Machine Learning is a core part of it. Start with introductory courses that cover:
    *   **What ML is:** Teaching computers to learn from data without being explicitly programmed.
    *   **Types of ML:** Supervised vs. Unsupervised Learning.
    *   **Simple Algorithms:** Linear Regression (for predictions) and K-Nearest Neighbors (for classification) are great starting points.

3.  **Online Resources are Your Best Friend:**
    *   **Coursera/edX:** Look for "Introduction to Machine Lear

In [12]:
text = """@LS_King: Yo why'd you blow up my car?! @ThugLife99: Cry about it #GetRekt"""
prompt = f"""Analyze this GTA Online tweet fight:
1. Who started it?
2. Most toxic word?
3. Suggested peace treaty (funny):
4. let your responses be short and avoid use of asterisks for displaying texxt
```{text}```"""

response = chat.send_message(prompt)
print(response.text)

Here's the analysis:

1.  LS_King started the tweet fight.
2.  GetRekt.
3.  ThugLife99 must buy LS_King a new car, then they both do a friendly stunt race where the loser buys pizza.


In [14]:
text = "Tesla is an electric vehicle company. Its stock price has increased by 20% in the last year. Apple is a technology company known for its iphones. Its stock price has increased by 15% in the last year. Amazon is an e-commerce and cloud computing giant. Its stock price has increased by 10% in the last year."
prompt = f"""Convert the following information into a well formatted table format with columns for company, industry and stock performance:
Text: ```{text}``` """

response = chat.send_message(prompt)
print(response.text)

| Company | Industry                  | Stock Performance      |
|---------|---------------------------|------------------------|
| Tesla   | Electric Vehicle          | Increased by 20%       |
| Apple   | Technology                | Increased by 15%       |
| Amazon  | E-commerce and Cloud Computing | Increased by 10%       |


In [16]:
text = "Artificial Intelligence is transforming industries like healthcare, finance, and education by improving efficiency and decision making."
prompt = f"""Format the response in a list:
- Summarize the key industries AI is transforming.
- Highlight its impact.
Text: ```{text}```

dont use asterisks"""

response = chat.send_message(prompt)
print(response.text)

- Key Industries AI is Transforming:
  - Healthcare
  - Finance
  - Education
- Highlight its Impact:
  - Improving efficiency
  - Enhancing decision making


In [18]:
text = "John is a data scientist with expertise in python and machine learning."
prompt = f"""Extract key details from the text and return the output in JSON format.
Text:
```{text}```
Output format:
{{
"name": ",
"profession":",
"skills": []
}}"""

response = chat.send_message(prompt)
print(response.text)

```json
{
"name": "John",
"profession": "data scientist",
"skills": ["python", "machine learning"]
}
```


In [20]:
prompt = "Write a structured paragraph with clear headings and subheadings about the impact of a balanced diet on physical and mental health."

response = chat.send_message(prompt)
print(response.text)

A balanced diet profoundly influences overall well-being, impacting both the body and mind significantly.

**Physical Health Benefits**
A well-rounded diet provides the necessary fuel for optimal bodily functions. This includes **Sustained Energy Levels**, as complex carbohydrates, healthy fats, and proteins ensure a steady release of energy throughout the day, preventing crashes and fatigue. Moreover, proper nutrition is crucial for **Disease Prevention**, strengthening the immune system, maintaining a healthy weight, and reducing the risk of chronic conditions such as heart disease, type 2 diabetes, and certain cancers.

**Mental Well-being**
Beyond the physical, a balanced diet plays a critical role in mental health. It supports **Mood Regulation** by providing essential nutrients that are precursors to neurotransmitters like serotonin and dopamine, which are vital for emotional stability and reducing symptoms of depression and anxiety. Furthermore, it enhances **Cognitive Function*